Notebook that pulls in data from wikipedia and the postal codes in Toronto,Ontario. With the paywall for geocoders, the lat/long data was provided via csv file. First data frame is merged with new data frame with geo coordinates. Map is generated from the final data frame


In [74]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import html5lib
import lxml.html as lh
from IPython.display import display

In [42]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r=requests.get(URL)  #gets the code from the wikipedia page


Takes the code from the page and pareses through, giving an html version of the page to read.

In [43]:
soup = BeautifulSoup(r.content, 'html.parser') 
#print(soup.prettify())    #Commented out due to the extremely long page generated by it. 

Takes the wikipedia page and uses xml to find the table then checks to make sure the column width is uniform throughout.

In [44]:
doc = lh.fromstring(r.content)
mytable = doc.xpath('//tr')
[len(T) for T in mytable[:12]] #checks to see if every line has the right number of columns

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

Takes the new table and finds the column headers

In [45]:
col = []
i=0

for t in mytable[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postal Code
"
2:"Borough
"
3:"Neighbourhood
"


Sorts the data into the table 

In [46]:
for j in range(1,len(mytable)):
    T=mytable[j]
    
    if len(T)!=3:
        break
    
    i=0
    
    for t in T.iterchildren():
        data=t.text_content()
        
        col[i][1].append(data)
        i+=1

In [47]:
[len(C) for (title,C) in col] #check to make sure each column is the same length

[181, 181, 181]

Multiple steps: takes the new table and pulls it into a dataframe, drops off extra '\n' in the lines, gets rid of lines with 'Not assigned' and drops the final row which did not have data.

In [48]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df=df.replace(r'\n','',regex=True)
df.columns = ['Postal Code', 'Borough', 'Neighbourhood']
df = df[df.Borough != 'Not assigned']
df.drop(df.tail(1).index,inplace=True)


In [49]:
df.shape

(103, 3)

////This is the end of part 1. Part 2 starts here //////


In [50]:
geo = pd.read_csv("GeoCoord.csv")  #pulls in the file with geo data

In [51]:
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [52]:
df2 = df

In [53]:
DF = pd.merge(df2, geo, how='left', on = 'Postal Code') #joins geo data frame with the original data frame by postal code

In [54]:
DF.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


////Part 3//////

In [55]:
import folium as fl

In [56]:
Lat = DF['Latitude']
Long = DF['Longitude']



In [75]:
map = fl.Map(location = [Lat.iloc[0], Long.iloc[0]], zoom_start = 12)

for i in range(0,len(DF)):
    fl.CircleMarker([Lat.iloc[i], Long.iloc[i]],radius = 5, color='blue',fill= True, fill_color='blue', popup=DF.iloc[i]['Neighbourhood']).add_to(map)

display(map)

Map shows each postal code and the popup labels show what neighbourhood(s) are inside each postal code. Observed more postal codes in downtown Toronto and that neighbourhoods are not based on postal codes.